In [1]:
from common.reward_function_testing.reward_mods import SF_EXP_4_Test_Modules
from common.utils import ExperimentRunner as expr
from common.dqn_envs import EXP_4_all_env_functions

from common.constants import*
# from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3 import DQN
import time

In [2]:
all_env_functions = EXP_4_all_env_functions()
all_env_functions

['make_dqn_env']

In [3]:
all_test_mods = SF_EXP_4_Test_Modules()
all_test_mods

EXP 4: Reward function modules with movement data and scaled rewards: ['SF_Default', 'SF_New_Movement_Data', 'SF_Default_Scaled_Rewards', 'SF_New_Movement_Data_Scaled_Rewards']

In [4]:
expr = expr(DQN, verbose=True)

running experiments on cpu


In [5]:
#Atari hyper params from RLZoo
#https://github.com/DLR-RM/rl-baselines3-zoo/blob/master/hyperparams/dqn.yml

# h_params = ( policy: 'CnnPolicy'
#   n_timesteps: !!float 1e7
#   buffer_size: 100000
#   learning_rate: !!float 1e-4
#   batch_size: 32
#   learning_starts: 100000
#   target_update_interval: 1000
#   train_freq: 4
#   gradient_steps: 1
#   exploration_fraction: 0.1
#   exploration_final_eps: 0.01,
#   seed: 2)

In [6]:
h_params = {'policy': ['CnnPolicy'],
  'buffer_size': [50_000],
  'learning_rate': [0.0001],
  'batch_size': [32],
  'learning_starts': {100000},
  'target_update_interval': [1000],
  'train_freq': [4],
  'gradient_steps': [1],
  'exploration_fraction': [0.1],
  'exploration_final_eps': [0.01],
  'seed': [2]}

In [7]:
from common.utils import build_all_permutations

In [8]:
h_params_list = build_all_permutations(h_params)
h_params_list

[{'policy': 'CnnPolicy',
  'buffer_size': 50000,
  'learning_rate': 0.0001,
  'batch_size': 32,
  'learning_starts': 100000,
  'target_update_interval': 1000,
  'train_freq': 4,
  'gradient_steps': 1,
  'exploration_fraction': 0.1,
  'exploration_final_eps': 0.01,
  'seed': 2}]

In [9]:
expr.set_tran_opts(total_timesteps=3_000_000, progress_bar=True)

In [10]:
expr.set_env_func_options(render_mode = None, n_procs = 4)
expr

expr: model: <class 'stable_baselines3.dqn.dqn.DQN'>,
base_env_class: None
h_parameters: None,
model_ops: {'tensorboard_log': './results/board/', 'verbose': 0},
model_train_ops: {'total_timesteps': 3000000, 'progress_bar': True, 'tb_log_name': 'DQN'},
env_builder opts: {'render_mode': None, 'n_procs': 4},
eval function: evaluate_policy,
eval function opts: {'deterministic': False},
device: cpu,
model current learning timesteps: 0

In [11]:
test = 1
total = len(all_test_mods.get_test_modules()) * len(all_env_functions.get_env_func_list()) * len(h_params_list)
total

4

In [12]:
from common.utils import cleanup_device

In [13]:
for module in all_test_mods.get_test_modules()[2:]:
    for env_func in all_env_functions.get_env_func_list():
        for h_param in h_params_list:
            
            policy = h_param['policy']
            test_name = f'module: {str(module.__name__)}, env_func: {env_func.__name__}, policy: {policy}'
            expr.set_base_env(module)
            expr.set_env_func(env_func)
            expr.set_tb_log_name(test_name)
            
            start = time.time()
            print(f'Test: {test} out of {total}')
            print(h_param)
            print(test_name)
            expr.train_model(total_timesteps=3_000_000, hyper_params=h_param)
            save_dir = f'{OPT_DIR}best_initial_models/{str(module.__name__)}_2_000_000_timesteps'
            expr.save_model(save_dir=save_dir)

            end = time.time()
            elapsed = round(((end - start) /60), 2)
            print(f'Total train time in minutes: {elapsed}')
            print()

            test += 1

            cleanup_device()


Test: 1 out of 4
{'policy': 'CnnPolicy', 'buffer_size': 50000, 'learning_rate': 0.0001, 'batch_size': 32, 'learning_starts': 100000, 'target_update_interval': 1000, 'train_freq': 4, 'gradient_steps': 1, 'exploration_fraction': 0.1, 'exploration_final_eps': 0.01, 'seed': 2}
module: SF_Default_Scaled_Rewards, env_func: make_dqn_env, policy: CnnPolicy
Learning...


Output()

Model Saved to: ./results/opt/best_initial_models/SF_Default_Scaled_Rewards_2_000_000_timesteps
Total train time in minutes: 109.62

Test: 2 out of 4
{'policy': 'CnnPolicy', 'buffer_size': 50000, 'learning_rate': 0.0001, 'batch_size': 32, 'learning_starts': 100000, 'target_update_interval': 1000, 'train_freq': 4, 'gradient_steps': 1, 'exploration_fraction': 0.1, 'exploration_final_eps': 0.01, 'seed': 2}
module: SF_New_Movement_Data_Scaled_Rewards, env_func: make_dqn_env, policy: CnnPolicy
Learning...


Output()

Model Saved to: ./results/opt/best_initial_models/SF_New_Movement_Data_Scaled_Rewards_2_000_000_timesteps
Total train time in minutes: 105.49

